In [8]:
import gdown
url='https://drive.google.com/uc?id=1gPFE1ozD6Y-M4fBa0dy4USdu426YMCB6'
filename='PtlibraryData(week7).csv'
gdown.download(url, filename, quiet = False)


#datafixing.py
import gdown
import pandas as pd

def dataCleaning(filename) :
    """
    : param filename: CSV 파일이름
    """
    df = pd.read_csv(filename, low_memory=False)
    df_book = df.dropna(axis=1, how='all')
    count_df = df_book[['도서명','저자','ISBN','권','대출건수']]
    loan_count = count_df.groupby(by=['도서명','저자','ISBN','권'], dropna=False).sum()
    dup_rows = df_book.duplicated(subset=['도서명','저자','ISBN','권'])
    unique_rows = ~dup_rows
    book_unique = df_book[unique_rows].copy()
    book_unique.set_index(['도서명','저자','ISBN','권'], inplace=True)
    book_unique.update(loan_count)
    book_return = book_unique.reset_index()
    book_return = book_return[df_book.columns]

    return book_return

import requests
from bs4 import BeautifulSoup

def dataFixing(df_book):
    """
    잘못된 값을 수정하거나 NaN을 채우는 함수
    :param df_book: dataCleaning() 함수에서 전처리된 데이터프레임
    """
    
    df_book = df_book.astype({'도서권수':'int32', '대출건수': 'int32'})
    set_isbn_na_rows = df_book['세트 ISBN'].isna()
    df_book.loc[set_isbn_na_rows, '세트 ISBN'] = ''

    # 발행년도 
    df_book2 = df_book.replace({'발행년도':'.*(\d{4}).*'}, r'\1', regex=True)
    unknown_year = df_book2['발행년도'].str.contains('\D', na=True)
    df_book2.loc[unknown_year, '발행년도'] = '-1'
    df_book2 = df_book2.astype({'발행년도': 'int32'})

    # 단군 력 빼기
    dangun_yy_rows = df_book2['발행년도'].gt(4000)
    df_book2.loc[dangun_yy_rows, '발행년도'] = df_book2.loc[dangun_yy_rows, '발행년도'] - 2333
    dangun_year = df_book2['발행년도'].gt(4000)
    df_book2.loc[dangun_year, '발행년도'] = -1

    # 0~1900년 사이
    old_books = df_book2['발행년도'].gt(0) & df_book2['발행년도'].lt(1900)
    df_book2.loc[old_books, '발행년도'] = -1

    na_rows = df_book2['도서명'].isna() | df_book2['저자'].isna() | df_book2['출판사'].isna() | df_book2['발행년도'].eq(-1)
    
    updated_sample = df_book2[na_rows].apply(get_book_info, axis=1, result_type = 'expand')

    df_book2 = df_book2.dropna(subset=['도서명', '저자', '출판사'])
    df_book2 = df_book2[df_book2['발행년도'] != -1]    

    return df_book2
    
def get_book_title(isbn):
    
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱

    title = soup.find('a', attrs={'class':'gd_name'}) \
            .get_text()
    return title

import re

def get_book_info(row):
    title = row['도서명']
    author = row['저자']
    pub = row['출판사']
    year = row['발행년도']

    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(row['ISBN']))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    try:
        if pd.isna(title):
            # 클래스 이름이 'gd_name'인 a 태그의 텍스트를 가져옵니다.
            title = soup.find('a', attrs={'class':'gd_name'}) \
                    .get_text()
    except AttributeError:
        pass

    try:
        if pd.isna(author):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            authors = soup.find('span', attrs={'class':'info_auth'}) \
                          .find_all('a')
            author_list = [auth.get_text() for auth in authors]
            author = ','.join(author_list)
    except AttributeError:
        pass
    
    try:
        if pd.isna(pub):
            # 클래스 이름이 'info_auth'인 span 태그 아래 a 태그의 텍스트를 가져옵니다.
            pub = soup.find('span', attrs={'class':'info_pub'}) \
                      .find('a') \
                      .get_text()
    except AttributeError:
        pass
    
    try:
        if year == -1:
            # 클래스 이름이 'info_date'인 span 태그 아래 텍스트를 가져옵니다.
            year_str = soup.find('span', attrs={'class':'info_date'}) \
                           .get_text()
            # 정규식으로 찾은 값 중에 첫 번째 것만 사용합니다.
            year = re.findall(r'\d{4}', year_str)[0]
    except AttributeError:
        pass

    return title, author, pub, year


link = ""
filename = 'PtlibraryData(week7).csv'
#url = f'https://drive.google.com/uc?id={link}'
directory = '/content/'

#gdown.download(url, filename, quiet=False)

pt_book_clean = dataCleaning(f"{directory}{filename}")
pt_book_clean.to_csv(f'{directory}pt_book_clean.csv', index=False)

#pt_book_fix = dataFixing(pt_book_clean)
#pt_book_fix.to_csv(f'{directory}pt_book_fix.csv', index=False)

Downloading...
From: https://drive.google.com/uc?id=1gPFE1ozD6Y-M4fBa0dy4USdu426YMCB6
To: /content/PtlibraryData(week7).csv
100%|██████████| 30.8M/30.8M [00:00<00:00, 160MB/s]


In [11]:
import pandas as pd
pt_book = pd.read_csv("pt_book_clean.csv", low_memory=False)
pt_book.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,2023,9.79117E+12,NaN,0,NaN,199.1,1,0.0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,2023,9.79113E+12,NaN,1,NaN,373.7,1,0.0,2023-03-30
2,3,(현직 고등학교 선생님들이 직접 고른) 생기부 필독서 100,주경아 외 지음,센시오,2023,9.79117E+12,NaN,1,NaN,376.6,1,0.0,2023-03-30
3,4,당연하게도 나는 너를,이꽃님 지음,우리학교,2023,9.79117E+12,NaN,4,NaN,813.7,1,0.0,2023-03-30
4,5,고요한 우연,김수빈 지음,문학동네,2023,9.78895E+12,NaN,0,NaN,813.7,1,0.0,2023-03-30


In [12]:
pt_book.isna().sum()

번호              0
도서명             0
저자             55
출판사          1829
발행년도           11
ISBN            0
세트 ISBN    193102
부가기호        49265
권          158389
주제분류번호        932
도서권수            0
대출건수            0
등록일자            0
dtype: int64

In [13]:
pt_book.loc[0,'도서권수'] = None
pt_book['도서권수'].isna().sum()

1

In [16]:
pt_book.loc[0,'도서권수']=1
pt_book = pt_book.astype({'도서권수':'int32','대출건수':'int32'})
pt_book.head(2)

,번호,도서명,저자,출판사,발행년도,...,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,2023,...,NaN,199.1,1,0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,2023,...,NaN,373.7,1,0,2023-03-30


In [18]:
set_isbn_na=pt_book['세트 ISBN'].isna()
pt_book.loc[set_isbn_na, '세트 ISBN']=''
pt_book.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,2023,9.79117E+12,,0,NaN,199.1,1,0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,2023,9.79113E+12,,1,NaN,373.7,1,0,2023-03-30
2,3,(현직 고등학교 선생님들이 직접 고른) 생기부 필독서 100,주경아 외 지음,센시오,2023,9.79117E+12,,1,NaN,376.6,1,0,2023-03-30
3,4,당연하게도 나는 너를,이꽃님 지음,우리학교,2023,9.79117E+12,,4,NaN,813.7,1,0,2023-03-30
4,5,고요한 우연,김수빈 지음,문학동네,2023,9.78895E+12,,0,NaN,813.7,1,0,2023-03-30


In [22]:
import numpy as np

pt_tmp = pt_book.replace(np.nan, '없음')
pt_tmp.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,2023,9.79117E+12,,0,없음,199.1,1,0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,2023,9.79113E+12,,1,없음,373.7,1,0,2023-03-30
2,3,(현직 고등학교 선생님들이 직접 고른) 생기부 필독서 100,주경아 외 지음,센시오,2023,9.79117E+12,,1,없음,376.6,1,0,2023-03-30
3,4,당연하게도 나는 너를,이꽃님 지음,우리학교,2023,9.79117E+12,,4,없음,813.7,1,0,2023-03-30
4,5,고요한 우연,김수빈 지음,문학동네,2023,9.78895E+12,,0,없음,813.7,1,0,2023-03-30


In [23]:
pt_book.replace({np.nan, '2023'},{'없음','23'}).head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,없음,9.79117E+12,,0,23,199.1,1,0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,없음,9.79113E+12,,1,23,373.7,1,0,2023-03-30
2,3,(현직 고등학교 선생님들이 직접 고른) 생기부 필독서 100,주경아 외 지음,센시오,없음,9.79117E+12,,1,23,376.6,1,0,2023-03-30
3,4,당연하게도 나는 너를,이꽃님 지음,우리학교,없음,9.79117E+12,,4,23,813.7,1,0,2023-03-30
4,5,고요한 우연,김수빈 지음,문학동네,없음,9.78895E+12,,0,23,813.7,1,0,2023-03-30


In [24]:
pt_book.replace({np.nan :'없음'},{'2023':'23'}).head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,세이노의 가르침,세이노 지음,데이원,2023,9.79117E+12,,0,NaN,199.1,1,0,2023-03-30
1,2,서울대생의 비밀과외,"소린TV,안소린 [공]지음",다산에듀,2023,9.79113E+12,,1,NaN,373.7,1,0,2023-03-30
2,3,(현직 고등학교 선생님들이 직접 고른) 생기부 필독서 100,주경아 외 지음,센시오,2023,9.79117E+12,,1,NaN,376.6,1,0,2023-03-30
3,4,당연하게도 나는 너를,이꽃님 지음,우리학교,2023,9.79117E+12,,4,NaN,813.7,1,0,2023-03-30
4,5,고요한 우연,김수빈 지음,문학동네,2023,9.78895E+12,,0,NaN,813.7,1,0,2023-03-30


In [25]:
pt_tmp=pt_book.replace({'부가기호':np.nan},'없음')
pt_tmp['부가기호'].isna().sum()

0

In [28]:
invalid_number=pt_book['발행년도'].str.contains('\n',na=True)
print(invalid_number.sum())
pt_book[invalid_number].head()

11


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
21082,22648,(월명스님의 희망꽃)사과 한 개의 행복,월명 지음,희망꽃,NaN,9.79119E+12,,0,NaN,814.7,1,0,2019-10-16
96794,100879,세계 역사를 만든 사람들,유훈 편,교학사,NaN,9.78891E+12,,NaN,NaN,990,1,6,2009-10-31
119305,128793,거짓말의 진화:자기정당화의 심리학,엘리엇 애런,추수밭,NaN,9.78899E+12,,0,NaN,181.8,1,42,2008-03-14
128194,138262,암·AIDS·마약은 이미 정복되었습니다,한동규 지음,유림,NaN,9.78899E+12,,1,NaN,513.994,1,2,2007-06-05
161841,174448,격암유록,김순열,대산,NaN,9.789E+12,9789000000000.0,NaN,상,187.3,1,8,2004-03-28


In [38]:
pt_book2=pt_book.replace({'발행년도':'.*(\d{4}).*'}, r'\1', regex=True)
pt_book2[invalid_number].head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
21082,22648,(월명스님의 희망꽃)사과 한 개의 행복,월명 지음,희망꽃,NaN,9.79119E+12,,0,NaN,814.7,1,0,2019-10-16
96794,100879,세계 역사를 만든 사람들,유훈 편,교학사,NaN,9.78891E+12,,NaN,NaN,990,1,6,2009-10-31
119305,128793,거짓말의 진화:자기정당화의 심리학,엘리엇 애런,추수밭,NaN,9.78899E+12,,0,NaN,181.8,1,42,2008-03-14
128194,138262,암·AIDS·마약은 이미 정복되었습니다,한동규 지음,유림,NaN,9.78899E+12,,1,NaN,513.994,1,2,2007-06-05
161841,174448,격암유록,김순열,대산,NaN,9.789E+12,9789000000000.0,NaN,상,187.3,1,8,2004-03-28


In [39]:
unknown_year=pt_book['발행년도'].str.contains('\0',na=True)
print(unknown_year.sum())
pt_book[unknown_year].head()

11


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
21082,22648,(월명스님의 희망꽃)사과 한 개의 행복,월명 지음,희망꽃,NaN,9.79119E+12,,0,NaN,814.7,1,0,2019-10-16
96794,100879,세계 역사를 만든 사람들,유훈 편,교학사,NaN,9.78891E+12,,NaN,NaN,990,1,6,2009-10-31
119305,128793,거짓말의 진화:자기정당화의 심리학,엘리엇 애런,추수밭,NaN,9.78899E+12,,0,NaN,181.8,1,42,2008-03-14
128194,138262,암·AIDS·마약은 이미 정복되었습니다,한동규 지음,유림,NaN,9.78899E+12,,1,NaN,513.994,1,2,2007-06-05
161841,174448,격암유록,김순열,대산,NaN,9.789E+12,9789000000000.0,NaN,상,187.3,1,8,2004-03-28
